# Critical Temperature of Superconductors

In [31]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor

from sklearn import preprocessing

from sklearn.decomposition import PCA

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import GridSearchCV

import os
from sklearn.metrics import r2_score, mean_squared_error

from utils import Step, Pipe, MultiplePipes, combination_already_tested, print_results

In [32]:
plt.style.use("seaborn-v0_8")

DATA_FOLDER = "data/"
OUTPUT_FOLDER = "outputs/"

In [33]:
RANDOM_STATE = 42

OUTLIER_REMOVAL = False

---
---
## Data Load

In [34]:
df = pd.concat(
    [
        pd.read_csv(DATA_FOLDER + "formula_train.csv").drop(columns=["critical_temp"]),
        pd.read_csv(DATA_FOLDER + "train.csv"),
    ],
    axis=1,
)
print("Shapes of Properties+Formula df: ", df.shape)

Shapes of Properties+Formula df:  (17010, 169)


In [35]:
# Remove "material" feature
df = df.drop(columns="material")

---
---
## Split

In [36]:
train, test = train_test_split(df, test_size=0.2, random_state=RANDOM_STATE)

X_train = train.drop(columns=["critical_temp"])
y_train = train[["critical_temp"]]

X_test = test.drop(columns=["critical_temp"])
y_test = test[["critical_temp"]]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13608, 167), (3402, 167), (13608, 1), (3402, 1))

---
---
## Remove Highly correlated features

In [37]:
# if REMOVE_HIGH_CORR_FEATURES:
#     corr_matrix = df.corr().abs()
#     upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
#     cols_to_drop = [column for column in upper.columns if any(upper[column] >= CORR_THRESHOLD)]

#     print("{} Cols Removed: {}".format(len(cols_to_drop), cols_to_drop))
#     X_train = X_train.drop(columns=cols_to_drop)
#     X_test = X_test.drop(columns=cols_to_drop)
#     print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [38]:
class FeaturesRemover:
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        corr_matrix = df.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        cols_to_drop = [column for column in upper.columns if any(upper[column] >= self.corr_threshold)]

        # print("{} Cols Removed: {}".format(len(cols_to_drop), cols_to_drop))
        X = X.drop(columns=cols_to_drop)
        return X

    def set_params(self, corr_threshold):
        self.corr_threshold = corr_threshold
        return self

---
---
## Outlier removal

In [39]:
if OUTLIER_REMOVAL:
    columns = train.columns
    outliers = pd.Series(index=train.index, dtype=bool)

    clf = LocalOutlierFactor(n_jobs=-1)
    # clf = IsolationForest(
    #     max_samples=1.0,
    #     contamination=0.001,
    #     n_jobs=-1,
    #     random_state=random_state,
    # )
    outliers = clf.fit_predict(train) == -1

    print("Outliers removed: {}".format(outliers.sum()))
    train = train[~outliers]

    X_train = train.drop(columns=["critical_temp"])
    y_train = train[["critical_temp"]]
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [40]:
# class OutliersRemover:
#     def __init__(self) -> None:
#         self.outliers_vector = None

#     def fit(self, X, y):
#         self.outliers_vector = pd.Series(index=X.index, dtype=bool)

#         clf = LocalOutlierFactor(n_jobs=-1)
#         # clf = IsolationForest(
#         #     max_samples=1.0,
#         #     contamination=0.001,
#         #     n_jobs=-1,
#         #     random_state=random_state,
#         # )
#         self.outliers_vector = clf.fit_predict(np.column_stack((X, y))) == -1
#         print("Outliers removed: {}".format(self.outliers_vector.sum()))
#         return self

#     def transform(self, X, y=None):

#         X = X[~self.outliers_vector]
#         y = y[~self.outliers_vector]

#         print("Outliers removed: {}".format(self.outliers_vector.sum()))
#         print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
#         return X, y

#     def set_params(self):
#         return self

---
---
## Preprocessing

In [41]:
features_remover_step = Step(
    "features_remover",
    FeaturesRemover(),
    {"corr_threshold": 0.95},
)
std_step = Step(
    "std",
    preprocessing.StandardScaler(),
)
minmax_step = Step(
    "minmax",
    preprocessing.MinMaxScaler(),
)
l1_step = Step(
    "l1",
    preprocessing.Normalizer(norm="l1"),
)
l2_step = Step(
    "l2",
    preprocessing.Normalizer(norm="l2"),
)
lmax_step = Step(
    "lmax",
    preprocessing.Normalizer(norm="max"),
)
pca_step = Step(
    "pca",
    PCA(random_state=RANDOM_STATE),
    {
        "n_components": [0.95],
        # "whiten": [True, False],
        # "svd_solver": "full",
    },
)

In [42]:
def grid_search(combinations: MultiplePipes, estimator_tag: str, save_results=True):

    # Iterate over *all* combinations
    for index, (pipeline, parameters, tag) in enumerate(combinations.combinations):
        print(
            "\nCombination {}/{}  |  {}\n\tParams {}".format(index + 1, len(combinations.combinations), tag, parameters)
        )

        # Check if this combination is already tested
        if save_results:
            file_name = OUTPUT_FOLDER + estimator_tag + "_output.csv"
            if combination_already_tested(file_name, parameters, tag):
                print("  ==> Already done. Skipped.")
                continue

        gs = GridSearchCV(
            estimator=pipeline,
            param_grid=parameters,
            n_jobs=-1,
            cv=3,
            verbose=0,
        )

        # Fit
        gs.fit(X_train, np.ravel(y_train))
        # Predict
        y_pred = gs.predict(X_test)
        # Test scores
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print("  ==> R2: {}\tMSE: {}".format(r2, mse))

        # Save results
        if save_results:
            outputs = pd.DataFrame(parameters or None, index=[0]).assign(tag=tag, MSE=mse, R2=r2)
            outputs = pd.concat(
                [pd.read_csv(file_name) if os.path.isfile(file_name) else pd.DataFrame(), outputs], axis=0
            )
            outputs.to_csv(file_name, index=False)

---
---
## Search

In [43]:
estimator_tag = "random_forest"
random_forest_step = Step(
    estimator_tag,
    RandomForestRegressor(n_jobs=-1, random_state=RANDOM_STATE),
    {
        "max_samples": [0.66],
        "criterion": ["squared_error"],
        "n_estimators": [200],
        "max_depth": [25],  # , 5, 10, 15, 20],
        "max_leaf_nodes": [None],  # 50, 100, 200, 300, 400
        "ccp_alpha": [0.0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 1.0],
        "max_features": [0.4],  # "sqrt", "log2", 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.9
    },
)

combinations = MultiplePipes(
    Pipe(random_forest_step),
    Pipe(minmax_step, random_forest_step),
    Pipe(minmax_step, pca_step, random_forest_step),
    Pipe(features_remover_step, minmax_step, pca_step, random_forest_step),
)
grid_search(combinations, estimator_tag=estimator_tag)

print_results(OUTPUT_FOLDER + estimator_tag + "_output.csv", 5)


Combination 1/28  |  random_forest
	Params {'random_forest__max_samples': [0.66], 'random_forest__criterion': ['squared_error'], 'random_forest__n_estimators': [200], 'random_forest__max_depth': [25], 'random_forest__max_leaf_nodes': [None], 'random_forest__ccp_alpha': [0.0], 'random_forest__max_features': [0.4]}
  ==> Already done. Skipped.

Combination 2/28  |  random_forest
	Params {'random_forest__max_samples': [0.66], 'random_forest__criterion': ['squared_error'], 'random_forest__n_estimators': [200], 'random_forest__max_depth': [25], 'random_forest__max_leaf_nodes': [None], 'random_forest__ccp_alpha': [1e-05], 'random_forest__max_features': [0.4]}
  ==> Already done. Skipped.

Combination 3/28  |  random_forest
	Params {'random_forest__max_samples': [0.66], 'random_forest__criterion': ['squared_error'], 'random_forest__n_estimators': [200], 'random_forest__max_depth': [25], 'random_forest__max_leaf_nodes': [None], 'random_forest__ccp_alpha': [0.0001], 'random_forest__max_feature

,tag,R2,MSE,random_forest__max_samples,random_forest__criterion,random_forest__n_estimators,random_forest__max_depth,random_forest__max_leaf_nodes,random_forest__ccp_alpha,random_forest__max_features,pca__n_components,features_remover__corr_threshold
55,minmax + random_forest,0.9268,83.2720,0.6600,squared_error,200,25,nan,0.0000,0.4000,nan,nan
54,minmax + random_forest,0.9268,83.2731,0.6600,squared_error,200,25,nan,0.0000,0.4000,nan,nan
56,minmax + random_forest,0.9268,83.2877,0.6600,squared_error,200,25,nan,0.0001,0.4000,nan,nan
57,minmax + random_forest,0.9267,83.4182,0.6600,squared_error,200,25,nan,0.0010,0.4000,nan,nan
16,minmax + random_forest,0.9264,83.7465,0.6600,squared_error,500,25,nan,0.0100,0.4000,nan,nan


In [44]:
estimator_tag = "linear_regression"
linear_regression_step = Step(
    estimator_tag,
    LinearRegression(n_jobs=-1),
)

combinations = MultiplePipes(
    Pipe(linear_regression_step),
    Pipe(minmax_step, linear_regression_step),
    Pipe(minmax_step, pca_step, linear_regression_step),
    Pipe(features_remover_step, minmax_step, pca_step, linear_regression_step),
)
grid_search(combinations, estimator_tag=estimator_tag)

print_results(OUTPUT_FOLDER + estimator_tag + "_output.csv", 5)


Combination 1/4  |  linear_regression
	Params {}
  ==> Already done. Skipped.

Combination 2/4  |  minmax + linear_regression
	Params {}
  ==> Already done. Skipped.

Combination 3/4  |  minmax + pca + linear_regression
	Params {'pca__n_components': [0.95]}
  ==> Already done. Skipped.

Combination 4/4  |  features_remover + minmax + pca + linear_regression
	Params {'features_remover__corr_threshold': [0.95], 'pca__n_components': [0.95]}
  ==> Already done. Skipped.


,tag,R2,MSE,pca__n_components,features_remover__corr_threshold
1,minmax + linear_regression,0.7538,280.1071,nan,nan
0,linear_regression,0.7538,280.1071,nan,nan
3,features_remover + minmax + pca + linear_regression,0.6677,378.1035,0.9500,0.9500
2,minmax + pca + linear_regression,0.6527,395.1441,0.9500,nan
